<div style="text-align: center; color: blue;">
    <h1>Visualisation, options llama on GPT2</h1>
</div>

**Nom :** BILLAUD Valère

**Date :** 22/05/2024

# 1) Global

## 1) Import

In [1]:
import pandas as pd 
import os
import warnings


## 2) Some function

In [2]:
def datanalyse(df, name_option):
    #Calcule mean_load_time
    mean_load_time = df['load_time'].str.replace("ms", "").astype(float).mean()
    #Calcule mean_sample_token_per_seconde
    mean_sample_token_per_seconde = df['token_per_sec'].astype(float).mean()
    #Calcule mean_prompt_eval_time_token_per_seconde
    mean_prompt_eval_time_token_per_seconde = df['token_per_sec.1'].astype(float).mean()
    #Calcule mean_eval_time_token_per_seconde
    mean_eval_time_token_per_seconde = df['token_per_sec.2'].astype(float).mean()
    #Calcule mean_total_time_token_per_seconde
    mean_total_time_token_per_seconde=0
    step1_mean_total_time_token_per_seconde= df['total_time'].str.replace("ms", "").str.replace("tokens","")
    step2_mean_total_time_token_per_seconde = [x.split("/") for x in step1_mean_total_time_token_per_seconde]
    step3_mean_total_time_token_per_seconde = [ 1000 / (float(x[0]) / float(x[1])) for x in step2_mean_total_time_token_per_seconde]
    for x in step3_mean_total_time_token_per_seconde :
        mean_total_time_token_per_seconde += x
    mean_total_time_token_per_seconde = mean_total_time_token_per_seconde / len(step3_mean_total_time_token_per_seconde)
   
    return [name_option,
            round(mean_load_time,3),
            round(mean_sample_token_per_seconde,3),
            round(mean_prompt_eval_time_token_per_seconde,3),
            round(mean_eval_time_token_per_seconde,3),
            round(mean_total_time_token_per_seconde,3)]

def stock(tab):
    to_write = ""
    for i in tab:
        to_write+= str(i) + ";"
    to_write = to_write[:-1]+"\n"
    with open('summerize.csv', 'a') as fichier:
        fichier.write(to_write)
        
def preparation_data(dataFrame, name_option):
    stock(datanalyse(dataFrame, name_option))


## 3) Merge data

In [3]:
colums= ["Option","mean_load_time","mean_sample_token_per_seconde","mean_prompt_eval_time_token_per_seconde","mean_eval_time_token_per_seconde","mean_total_time_token_per_seconde"]
if os.path.exists("summerize.csv"):
    os.remove("summerize.csv")
stock(colums)


In [4]:
# Preaparing data for gpt2_test_input_only_aggregation.csv
preparation_data(pd.read_csv("gpt2_test_input_only_aggregation.csv", sep=';', encoding='latin1'), "without option")

In [5]:
# Preaparing data for gpt2_test_input_ctk_ctv_aggregation.csv
def f32(string):
    return "_f32_" in string
def f16(string):
    return "_f16_" in string
#option type f32 in cache for K and V
df = pd.read_csv("gpt2_test_input_ctv_ctk_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_ctv_ctk_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(f32)]
name_option = "KV_f32"
preparation_data(df,name_option)

#option type f16 in cache for K and V
df = pd.read_csv("gpt2_test_input_ctv_ctk_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_ctv_ctk_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(f16)]
name_option = "KV_f16"
preparation_data(df,name_option)


In [6]:
# Preaparing data for gpt2_test_input_grp_att_aggregation.csv

def n(string):
    return "att_n_" in string
def w(string):
    return "att_w_" in string
def nw(string):
    return "att_w_n" in string

#option group attention factor 2 (default 1)
df = pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(n)]
name_option = "grp_att_factor_2"
preparation_data(df,name_option)

#option groupe attention width 256 (default 256)
df = pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(w)]
name_option = "grp_att_width_256"
preparation_data(df,name_option)

#option group attention factore 2 and groupe attention width 256
df = pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_grp_att_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(nw)]
name_option = "grp_att_factor_width"
preparation_data(df,name_option)

In [7]:
# Preaparing data for gpt2_test_input_keep_aggregation.csv

def zero(string):
    return "_0_" in string
def twenty(string):
    return "_20_" in string
def all_(string):
    return "_all_" in string

#option number of tokens to keep from the initial prompt 0 (default : 0)
df = pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(zero)]
name_option = "keep_0"
preparation_data(df,name_option)

#option number of tokens to keep from the initial prompt 20 (default : 0)
df = pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(twenty)]
name_option = "keep_20"
preparation_data(df,name_option)

#option number of tokens to keep from the initial prompt -1 (all) (default : 0)
df = pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_keep_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(all_)]
name_option = "keep_all"
preparation_data(df,name_option)

In [8]:
# Preaparing data for gpt2_test_input_min-p_aggregation.csv

def one(string):
    return "_01_" in string
def ten(string):
    return "_05_" in string
def thirty(string):
    return "_99_" in string

#option min probability sampling 0.01 (default : 0.1)
df = pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(one)]
name_option = "min-p 0.01"
preparation_data(df,name_option)

#option min probability sampling 0.1 (default : 0.1)
df = pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(ten)]
name_option = "min-p 0.1"
preparation_data(df,name_option)

#option min probability sampling 0.3 (default : 0.1)
df = pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')[pd.read_csv("gpt2_test_input_min-p_aggregation.csv", sep=';', encoding='latin1')['name_'].apply(thirty)]
name_option = "min-p 0.3"
preparation_data(df,name_option)

In [9]:
# Preaparing data for gpt2_test_input_mirostat_aggregation.csv

def _mirostat1_br002_ent15(string):
    return "_mirostat1_br002_ent15" in string
def _mirostat1_br002_ent2(string):
    return "_mirostat1_br002_ent2" in string
def _mirostat1_br002_ent5(string):
    return "_mirostat1_br002_ent5" in string
def _mirostat1_br002_ent8(string):
    return "_mirostat1_br002_ent8" in string

def _mirostat1_br02_ent15(string):
    return "_mirostat1_br02_ent15" in string
def _mirostat1_br02_ent2(string):
    return "_mirostat1_br02_ent2" in string
def _mirostat1_br02_ent5(string):
    return "_mirostat1_br02_ent5" in string
def _mirostat1_br02_ent8(string):
    return "_mirostat1_br02_ent8" in string

def _mirostat1_br05_ent15(string):
    return "_mirostat1_br05_ent15" in string
def _mirostat1_br05_ent2(string):
    return "_mirostat1_br05_ent2" in string
def _mirostat1_br05_ent5(string):
    return "_mirostat1_br05_ent5" in string
def _mirostat1_br05_ent8(string):
    return "_mirostat1_br05_ent8" in string


def _mirostat2_br002_ent15(string):
    return "_mirostat2_br002_ent15" in string
def _mirostat2_br002_ent2(string):
    return "_mirostat2_br002_ent2" in string
def _mirostat2_br002_ent5(string):
    return "_mirostat2_br002_ent5" in string
def _mirostat2_br002_ent8(string):
    return "_mirostat2_br002_ent8" in string

def _mirostat2_br02_ent15(string):
    return "_mirostat2_br02_ent15" in string
def _mirostat2_br02_ent2(string):
    return "_mirostat2_br02_ent2" in string
def _mirostat2_br02_ent5(string):
    return "_mirostat2_br02_ent5" in string
def _mirostat2_br02_ent8(string):
    return "_mirostat2_br02_ent8" in string

def _mirostat2_br05_ent15(string):
    return "_mirostat2_br05_ent15" in string
def _mirostat2_br05_ent2(string):
    return "_mirostat2_br05_ent2" in string
def _mirostat2_br05_ent5(string):
    return "_mirostat2_br05_ent5" in string
def _mirostat2_br05_ent8(string):
    return "_mirostat2_br05_ent8" in string

csv = "gpt2_test_input_mirostat_aggregation.csv"
#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br002_ent15)]
name_option = "mirostat 1 / mirostat-lr 0.02 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br002_ent2)]
name_option = "mirostat 1 / mirostat-lr 0.02 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br002_ent5)]
name_option = "mirostat 1 / mirostat-lr 0.02 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br002_ent8)]
name_option = "mirostat 1 / mirostat-lr 0.02 / mirostat-ent 8"
preparation_data(df,name_option)


#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br02_ent15)]
name_option = "mirostat 1 / mirostat-lr 0.2 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br02_ent2)]
name_option = "mirostat 1 / mirostat-lr 0.2 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br02_ent5)]
name_option = "mirostat 1 / mirostat-lr 0.2 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br02_ent8)]
name_option = "mirostat 1 / mirostat-lr 0.2 / mirostat-ent 8"
preparation_data(df,name_option)


#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br05_ent15)]
name_option = "mirostat 1 / mirostat-lr 0.5 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br05_ent2)]
name_option = "mirostat 1 / mirostat-lr 0.5 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br05_ent5)]
name_option = "mirostat 1 / mirostat-lr 0.5 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V1 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat1_br05_ent8)]
name_option = "mirostat 1 / mirostat-lr 0.5 / mirostat-ent 8"
preparation_data(df,name_option)



#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br002_ent15)]
name_option = "mirostat 2 / mirostat-lr 0.02 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br002_ent2)]
name_option = "mirostat 2 / mirostat-lr 0.02 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br002_ent5)]
name_option = "mirostat 2 / mirostat-lr 0.02 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.02 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br002_ent8)]
name_option = "mirostat 2 / mirostat-lr 0.02 / mirostat-ent 8"
preparation_data(df,name_option)


#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br02_ent15)]
name_option = "mirostat 2 / mirostat-lr 0.2 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br02_ent2)]
name_option = "mirostat 2 / mirostat-lr 0.2 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br02_ent5)]
name_option = "mirostat 2 / mirostat-lr 0.2 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.2 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br02_ent8)]
name_option = "mirostat 2 / mirostat-lr 0.2 / mirostat-ent 8"
preparation_data(df,name_option)


#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 15 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br05_ent15)]
name_option = "mirostat 2 / mirostat-lr 0.5 / mirostat-ent 15"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 2 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br05_ent2)]
name_option = "mirostat 2 / mirostat-lr 0.5 / mirostat-ent 2"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 5 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br05_ent5)]
name_option = "mirostat 2 / mirostat-lr 0.5 / mirostat-ent 5"
preparation_data(df,name_option)

#option mirostat : V2 (default : not use) 
#option mirostat learning rate : 0.5 (default : 0.1)
#option mirostat entropy : 8 (default : 5)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_mirostat2_br05_ent8)]
name_option = "mirostat 2 / mirostat-lr 0.5 / mirostat-ent 8"
preparation_data(df,name_option)

In [10]:
# Preaparing data for gpt2_test_input_nkvo_aggregation.csv
def nkvo(string):
    return "nkvo" in string
def not_nkvo(string):
    return not nkvo(string)

csv="gpt2_test_input_nkvo_aggregation.csv"

#option disable KV offload
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(nkvo)]
name_option = "disable KV offload"
preparation_data(df,name_option)

#without option
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(not_nkvo)]
name_option = "Normal"
preparation_data(df,name_option)


In [11]:
# Preaparing data for gpt2_test_input_repeat_last_n_aggregation.csv
def _0(string):
    return "_0_" in string
def _32(string):
    return "_32_" in string
def _128(string):
    return "_128_" in string
def _all(string):
    return "_all_" in string

csv="gpt2_test_input_repeat_last_n_aggregation.csv"

#option last n tokens to consider for penalize : 0 (default : 64)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_0)]
name_option = "repeat last n : 0"
preparation_data(df,name_option)

#option last n tokens to consider for penalize : 32 (default : 64)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_32)]
name_option = "repeat last n : 32"
preparation_data(df,name_option)

#option last n tokens to consider for penalize : 128 (default : 64)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_128)]
name_option = "repeat last n : 128"
preparation_data(df,name_option)

#option last n tokens to consider for penalize : context file (default : 64)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_all)]
name_option = "repeat last n : context file"
preparation_data(df,name_option)

In [12]:
# Preaparing data for gpt2_test_input_repeat_penalty_aggregation.csv
def _1(string):
    return "_1_" in string
def _1_5(string):
    return "_1.5_" in string
def _20(string):
    return "_20_" in string


csv="gpt2_test_input_repeat_penalty_aggregation.csv"

#option penalize repeat sequence of tokens : 1 (default : 1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_1)]
name_option = "repeat penalty : 1"
preparation_data(df,name_option)

#option penalize repeat sequence of tokens : 1.5 (default : 1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_1_5)]
name_option = "repeat penalty : 1.5"
preparation_data(df,name_option)

#option penalize repeat sequence of tokens : 20 (default : 1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_20)]
name_option = "repeat penalty n : 20"
preparation_data(df,name_option)



In [13]:
# Preaparing data for gpt2_test_input_split_aggregation.csv
def _np01_ns10_ps10_(string):
    return "_np01_ns10_ps10_" in string
def _np01_ns10_ps1_(string):
    return "_np01_ns10_ps1_" in string
def _np01_ns10_ps2_(string):
    return "_np01_ns10_ps2_" in string
def _np01_ns1_ps10(string):
    return "_np01_ns1_ps10_" in string
def _np01_ns1_ps1_(string):
    return "_np01_ns1_ps1_" in string
def _np01_ns1_ps2_(string):
    return "_np01_ns1_ps2_" in string
def _np01_ns2_ps10_(string):
    return "_np01_ns2_ps10_" in string
def _np01_ns2_ps1_(string):
    return "_np01_ns2_ps1_" in string
def _np01_ns2_ps2_(string):
    return "_np01_ns2_ps2_" in string
def _np05_ns10_ps10_(string):
    return "_np05_ns10_ps10_" in string
def _np05_ns10_ps1_(string):
    return "_np05_ns10_ps1_" in string
def _np05_ns10_ps2_(string):
    return "_np05_ns10_ps2_" in string
def _np05_ns1_ps10_(string):
    return "_np05_ns1_ps10_" in string
def _np05_ns1_ps1_(string):
    return "_np05_ns1_ps1_" in string
def _np05_ns1_ps2_(string):
    return "_np05_ns1_ps2_" in string
def _np05_ns2_ps10_(string):
    return "_np05_ns2_ps10_" in string
def _np05_ns2_ps1_(string):
    return "_np05_ns2_ps1_" in string
def _np05_ns2_ps2_(string):
    return "_np05_ns2_ps2_" in string
def _np09_ns10_ps10(string):
    return "_np09_ns10_ps10" in string
def _np09_ns10_ps1_(string):
    return "_np09_ns10_ps1_" in string
def _np09_ns10_ps2_(string):
    return "_np09_ns10_ps2_" in string
def _np09_ns1_ps10_(string):
    return "_np09_ns1_ps10_" in string
def _np09_ns1_ps1_(string):
    return "_np09_ns1_ps1_" in string
def _np09_ns1_ps2_(string):
    return "_np09_ns1_ps2_" in string
def _np09_ns2_ps10_(string):
    return "_np09_ns2_ps10_" in string
def _np09_ns2_ps1(string):
    return "_np09_ns2_ps1" in string
def _np09_ns2_ps2(string):
    return "_np09_ns2_ps2" in string

csv="gpt2_test_input_split_aggregation.csv"

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns10_ps10_)]
name_option = "_np01_ns10_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns10_ps1_)]
name_option = "_np01_ns10_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns10_ps2_)]
name_option = "_np01_ns10_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns1_ps10)]
name_option = "_np01_ns1_ps10"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns1_ps1_)]
name_option = "_np01_ns1_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns1_ps2_)]
name_option = "_np01_ns1_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns2_ps10_)]
name_option = "_np01_ns2_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns2_ps1_)]
name_option = "_np01_ns2_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np01_ns2_ps2_)]
name_option = "_np01_ns2_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns10_ps10_)]
name_option = "_np05_ns10_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns10_ps1_)]
name_option = "_np05_ns10_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns10_ps2_)]
name_option = "_np05_ns10_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns1_ps10_)]
name_option = "_np05_ns1_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns1_ps1_)]
name_option = "_np05_ns1_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns1_ps2_)]
name_option = "_np05_ns1_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns2_ps10_)]
name_option = "_np05_ns2_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns2_ps1_)]
name_option = "_np05_ns2_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np05_ns2_ps2_)]
name_option = "_np05_ns2_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns10_ps10)]
name_option = "_np09_ns10_ps10"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns10_ps1_)]
name_option = "_np09_ns10_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns10_ps2_)]
name_option = "_np09_ns10_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns1_ps10_)]
name_option = "_np09_ns1_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns1_ps1_)]
name_option = "_np09_ns1_ps1_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns1_ps2_)]
name_option = "_np09_ns1_ps2_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns2_ps10_)]
name_option = "_np09_ns2_ps10_"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns2_ps1)]
name_option = "_np09_ns2_ps1"
preparation_data(df,name_option)

#option   -np N, --parallel N   number of parallel sequences to decode (default: 1)
#option   -ns N, --sequences N  number of sequences to decode (default: 1)
#option   -ps N, --p-split N    speculative decoding split probability (default: 0.1)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_np09_ns2_ps2)]
name_option = "_np09_ns2_ps2"
preparation_data(df,name_option)



In [14]:
# Preaparing data for gpt2_test_input_temp_aggregation.csv
def _0(string):
    return "0_" in string
def _0_5(string):
    return "05_" in string
def _5(string):
    return "5_" in string


csv="gpt2_test_input_temp_aggregation.csv"

#option temp : 0 (default : 0.8)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_0)]
name_option = "temp : 0"
preparation_data(df,name_option)

#option temp : 0.5 (default : 0.8)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_0_5)]
name_option = "temp : 0.5"
preparation_data(df,name_option)

#option temp : 5 (default : 0.8)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_5)]
name_option = "temp : 5"
preparation_data(df,name_option)

In [15]:
# Preaparing data for gpt2_test_input_tfs_aggregation.csv
def _10(string):
    return "_10_" in string
def _60(string):
    return "_60_" in string
def _80(string):
    return "_80_" in string
def _90(string):
    return "_90_" in string
def _95(string):
    return "_95_" in string
def _100(string):
    return "_100_" in string


csv="gpt2_test_input_tfs_aggregation.csv"

#option tail free sampling : 0.1 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_10)]
name_option = "tfs : 0.1"
preparation_data(df,name_option)

#option tail free sampling : 0.6 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_60)]
name_option = "tfs : 0.6"
preparation_data(df,name_option)

#option tail free sampling : 0.8 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_80)]
name_option = "tfs : 0.8"
preparation_data(df,name_option)

#option tail free sampling : 0.9 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_90)]
name_option = "tfs : 0.9"
preparation_data(df,name_option)

#option tail free sampling : 0.95 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_95)]
name_option = "tfs : 0.95"
preparation_data(df,name_option)

#option tail free sampling : & (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_100)]
name_option = "tfs : 1 disabled"
preparation_data(df,name_option)

In [16]:
# Preaparing data for gpt2_test_input_thraed_aggregation.csv
def _1(string):
    return "_thread1_" in string
def _2(string):
    return "_thread2_" in string
def _4(string):
    return "_thread4_" in string
def _8(string):
    return "_thread8_" in string

csv = "gpt2_test_input_thraed_aggregation.csv"

#option Threads : 1 (default : 4)
#option Threads : 1 (default : same Threads)
#option Threads : 1 (default : same Threads)
#option Threads : 1 (default : same Threads)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_1)]
name_option = "Threads 1"
preparation_data(df,name_option)

#option Threads : 2 (default : 4)
#option Threads : 2 (default : same Threads)
#option Threads : 2 (default : same Threads)
#option Threads : 2 (default : same Threads)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_2)]
name_option = "Threads 2"
preparation_data(df,name_option)

#option Threads : 4 (default : 4)
#option Threads : 4 (default : same Threads)
#option Threads : 4 (default : same Threads)
#option Threads : 4 (default : same Threads)
df = pd.read_csv(csv, sep=';', encoding='latin4')[pd.read_csv(csv, sep=';', encoding='latin4')['name_'].apply(_4)]
name_option = "Threads 4"
preparation_data(df,name_option)

#option Threads : 8 (default : 4)
#option Threads : 8 (default : same Threads)
#option Threads : 8 (default : same Threads)
#option Threads : 8 (default : same Threads)
df = pd.read_csv(csv, sep=';', encoding='latin8')[pd.read_csv(csv, sep=';', encoding='latin8')['name_'].apply(_8)]
name_option = "Threads 8"
preparation_data(df,name_option)

In [17]:
# Preaparing data for gpt2_test_input_top-k_aggregation.csv
def _5(string):
    return "_5_" in string
def _20(string):
    return "_20_" in string
def _100(string):
    return "_100_" in string


csv="gpt2_test_input_top-k_aggregation.csv"

#option top-k sampling : 5 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_5)]
name_option = "top-k : 5 "
preparation_data(df,name_option)

#option top-k sampling : 20 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_20)]
name_option = "top-k : 20 "
preparation_data(df,name_option)

#option top-k sampling : 100 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_100)]
name_option = "top-k : 100 "
preparation_data(df,name_option)

In [18]:
# Preaparing data for gpt2_test_input_top-p_aggregation.csv
def _01(string):
    return "_01_" in string
def _05(string):
    return "_05_" in string
def _99(string):
    return "_99_" in string


csv="gpt2_test_input_top-p_aggregation.csv"

#option top-k sampling : 0.1 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_01)]
name_option = "top-p : 5 "
preparation_data(df,name_option)

#option top-k sampling : 0.5 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_05)]
name_option = "top-p : 20 "
preparation_data(df,name_option)

#option top-k sampling : 0.99 (default : 40)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_99)]
name_option = "top-k : 100 "
preparation_data(df,name_option)

In [19]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv
def _0(string):
    return "_0_" in string
def _5(string):
    return "_5_" in string
def _9(string):
    return "_9_" in string


csv="gpt2_test_input_typical_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_0)]
name_option = "typicaly : 0 "
preparation_data(df,name_option)

#option locally typical sampling: 5 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_5)]
name_option = "typicaly : 5 "
preparation_data(df,name_option)

#option locally typical sampling: 9 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')[pd.read_csv(csv, sep=';', encoding='latin1')['name_'].apply(_9)]
name_option = "typicaly : 9 "
preparation_data(df,name_option)

In [20]:
csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')

In [21]:
df

,Option,mean_load_time,mean_sample_token_per_seconde,mean_prompt_eval_time_token_per_seconde,mean_eval_time_token_per_seconde,mean_total_time_token_per_seconde
0,without option,943.806,10216.819,19.284,3.827,4.454
1,KV_f32,330.722,12319.718,32.130,11.391,12.766
2,KV_f16,313.495,12124.155,35.558,12.617,14.311
3,grp_att_factor_2,165.125,12501.543,35.580,13.423,14.930
4,grp_att_width_256,302.728,12501.669,37.633,14.262,15.816
...,...,...,...,...,...,...
89,top-p : 20,508.335,12253.822,31.408,11.621,12.724
90,top-k : 100,424.816,12116.249,29.658,13.351,14.357
91,typicaly : 0,661.316,11811.744,35.932,14.107,16.039
92,typicaly : 5,308.366,11734.240,31.768,17.026,18.205


## 4) Load_time

In [22]:
csv = "summerize.csv"
df_load_time = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_load_time']]

In [23]:
df_load_timedf_sorted = df_load_time.sort_values(by='mean_load_time')
for i in range(len(df_load_timedf_sorted)):
    print(df_load_timedf_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_load_timedf_sorted.iloc[i,1]) + "  ms")

typicaly : 9      with a mean time of   141.671  ms
Threads 4     with a mean time of   155.05  ms
grp_att_factor_2     with a mean time of   165.125  ms
mirostat 2 / mirostat-lr 0.2 / mirostat-ent 15     with a mean time of   171.038  ms
mirostat 2 / mirostat-lr 0.2 / mirostat-ent 5     with a mean time of   173.982  ms
Threads 2     with a mean time of   181.132  ms
min-p 0.1     with a mean time of   182.367  ms
_np01_ns1_ps10     with a mean time of   213.979  ms
min-p 0.3     with a mean time of   222.327  ms
Threads 1     with a mean time of   225.786  ms
_np05_ns2_ps1_     with a mean time of   245.314  ms
_np05_ns2_ps10_     with a mean time of   261.988  ms
mirostat 2 / mirostat-lr 0.02 / mirostat-ent 15     with a mean time of   284.98  ms
min-p 0.01     with a mean time of   289.35  ms
grp_att_width_256     with a mean time of   302.728  ms
_np09_ns10_ps1_     with a mean time of   303.838  ms
mirostat 2 / mirostat-lr 0.5 / mirostat-ent 5     with a mean time of   306.154  m

## 5) sample token per sec

In [24]:
csv = "summerize.csv"
df_sample = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_sample_token_per_seconde']]

In [25]:
df_sample_sorted = df_sample.sort_values(by='mean_sample_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_sample_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_sample_sorted.iloc[i,1]) + "  tokens per seconde")

top-k : 5      with a mean time of   14635.826  tokens per seconde
Threads 1     with a mean time of   13850.154  tokens per seconde
top-k : 20      with a mean time of   13011.292  tokens per seconde
top-p : 5      with a mean time of   12938.256  tokens per seconde
_np05_ns2_ps1_     with a mean time of   12868.266  tokens per seconde
Threads 4     with a mean time of   12597.441  tokens per seconde
Threads 2     with a mean time of   12557.622  tokens per seconde
grp_att_width_256     with a mean time of   12501.669  tokens per seconde
grp_att_factor_2     with a mean time of   12501.543  tokens per seconde
repeat last n : 32     with a mean time of   12481.903  tokens per seconde
grp_att_factor_width     with a mean time of   12474.818  tokens per seconde
KV_f32     with a mean time of   12319.718  tokens per seconde
top-p : 20      with a mean time of   12253.822  tokens per seconde
repeat last n : 128     with a mean time of   12133.858  tokens per seconde
KV_f16     with a mean 

## 6) mean_prompt_eval_time_token_per_seconde

In [26]:
csv = "summerize.csv"
df_prompt_eval = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_prompt_eval_time_token_per_seconde']]

In [27]:
df_prompt_eval_sorted = df_prompt_eval.sort_values(by='mean_prompt_eval_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_sample_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_sample_sorted.iloc[i,1]) + "  tokens per seconde")

top-k : 5      with a mean time of   14635.826  tokens per seconde
Threads 1     with a mean time of   13850.154  tokens per seconde
top-k : 20      with a mean time of   13011.292  tokens per seconde
top-p : 5      with a mean time of   12938.256  tokens per seconde
_np05_ns2_ps1_     with a mean time of   12868.266  tokens per seconde
Threads 4     with a mean time of   12597.441  tokens per seconde
Threads 2     with a mean time of   12557.622  tokens per seconde
grp_att_width_256     with a mean time of   12501.669  tokens per seconde
grp_att_factor_2     with a mean time of   12501.543  tokens per seconde
repeat last n : 32     with a mean time of   12481.903  tokens per seconde
grp_att_factor_width     with a mean time of   12474.818  tokens per seconde
KV_f32     with a mean time of   12319.718  tokens per seconde
top-p : 20      with a mean time of   12253.822  tokens per seconde
repeat last n : 128     with a mean time of   12133.858  tokens per seconde
KV_f16     with a mean 

## 7) mean_eval_time_token_per_seconde

In [28]:
csv = "summerize.csv"
df_eval = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_eval_time_token_per_seconde']]

In [29]:
df_eval_sorted = df_eval.sort_values(by='mean_eval_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_eval_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_eval_sorted.iloc[i,1]) + "  tokens per seconde")

Threads 2     with a mean time of   25.434  tokens per seconde
Threads 1     with a mean time of   20.29  tokens per seconde
typicaly : 9      with a mean time of   17.19  tokens per seconde
typicaly : 5      with a mean time of   17.026  tokens per seconde
_np05_ns2_ps1_     with a mean time of   15.691  tokens per seconde
Threads 4     with a mean time of   14.777  tokens per seconde
grp_att_width_256     with a mean time of   14.262  tokens per seconde
typicaly : 0      with a mean time of   14.107  tokens per seconde
grp_att_factor_width     with a mean time of   13.596  tokens per seconde
top-p : 5      with a mean time of   13.569  tokens per seconde
grp_att_factor_2     with a mean time of   13.423  tokens per seconde
top-k : 100      with a mean time of   13.351  tokens per seconde
KV_f16     with a mean time of   12.617  tokens per seconde
top-k : 100      with a mean time of   12.544  tokens per seconde
repeat last n : 32     with a mean time of   11.882  tokens per seconde
t

## 8) mean_total_time_token_per_seconde

In [30]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]

In [31]:
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of   14.807  tokens per seconde
top-k : 100      with a mean time of   14.357  tokens per seconde
KV_f16     with a mean time of   14.311  tokens per seconde
top-k : 100      with a mean time of   13.679  tokens per seconde
repeat last n : 32     with a mean time of   13.253  tokens per seconde


## Try new commandes and options 

### Try1

 ``` ../llama.cpp/main.exe -m "$model" -s 987654321 --file "$input_file" -n 100 -t 2 --typical 0.9 -np 0.5 -ns 2 -ps 1 -gaw 256 --top-p 5 -ctk f16 -ctv f16 --top-k 100 --repeat_last_n 32 --temp 0 --tfs 0.95 > "$output_res_file" 2> "$output_log_file" ```

In [32]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv

csv="gpt2_test_input_try1_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')
name_option = "try1 "
preparation_data(df,name_option)



In [33]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

try1      with a mean time of   29.265  tokens per seconde
Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of   14.807  tokens per seconde
top-k : 100      with a mean time of   14.357  tokens per seconde
KV_f16     with a mean time of   14.311  tokens per seconde
top-k : 100      with a mean time of   13.679  tokens per seconde
repeat last n

### Try2 

```../llama.cpp/main.exe -m "$model" -s 987654321 --file "$input_file" -n 100 -t 2 --typical 0.9 -np 0.5 -ns 2 -ps 1 -gaw 256 --top-p 5 -ctk f16 -ctv f16 --top-k 100 --repeat_last_n 32 > "$output_res_file" 2> "$output_log_file"```

In [34]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv

csv="gpt2_test_input_try2_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')
name_option = "try2 "
preparation_data(df,name_option)

In [35]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

try1      with a mean time of   29.265  tokens per seconde
try2      with a mean time of   28.525  tokens per seconde
Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of   14.807  tokens per seconde
top-k : 100      with a mean time of   14.357  tokens per seconde
KV_f16     with a mean time of   14.311  tokens per seconde
top-k : 100      wit

### Try3

``` 
../llama.cpp/main.exe -m "$model" -s 987654321 --file "$input_file" -n 100 -t 2 --typical 0.9 -gaw 256 --top-p 5 -ctk f16 -ctv f16 --top-k 100 --repeat_last_n 32 > "$output_res_file" 2> "$output_log_file"
```

In [36]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv

csv="gpt2_test_input_try3_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')
name_option = "try3 "
preparation_data(df,name_option)

In [37]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

try1      with a mean time of   29.265  tokens per seconde
try3      with a mean time of   28.999  tokens per seconde
try2      with a mean time of   28.525  tokens per seconde
Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of   14.807  tokens per seconde
top-k : 100      with a mean time of   14.357  tokens per seconde
KV_f16     with a mea

### Try 4

```
../llama.cpp/main.exe -m "$model" -s 987654321 --file "$input_file" -n 100 -t 2 --typical 0.9  --repeat_last_n 32 > "$output_res_file" 2> "$output_log_file"
```

In [38]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv

csv="gpt2_test_input_try4_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')
name_option = "try4 "
preparation_data(df,name_option)

In [39]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

try1      with a mean time of   29.265  tokens per seconde
try3      with a mean time of   28.999  tokens per seconde
try2      with a mean time of   28.525  tokens per seconde
try4      with a mean time of   28.016  tokens per seconde
Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of   14.807  tokens per seconde
top-k : 100      with a mean

### Try 5

```
../llama.cpp/main.exe -m "$model" -s 987654321 --file "$input_file" -n 100 -t 2 --typical 0.9 > "$output_res_file" 2> "$output_log_file"

```

In [40]:
# Preaparing data for gpt2_test_input_typical_aggregation.csv

csv="gpt2_test_input_try5_aggregation.csv"

#option locally typical sampling: 0 (default : 1 disabled)
df = pd.read_csv(csv, sep=';', encoding='latin1')
name_option = "try5"
preparation_data(df,name_option)

In [41]:
csv = "summerize.csv"
df_total = pd.read_csv(csv, sep=';', encoding='latin1')[['Option','mean_total_time_token_per_seconde']]
df_total_sorted = df_total.sort_values(by='mean_total_time_token_per_seconde' , ascending=False)
for i in range(len(df_sample_sorted)):
    print(df_total_sorted.iloc[i,0][:] + "     with a mean time of   " + str(df_total_sorted.iloc[i,1]) + "  tokens per seconde")

try1      with a mean time of   29.265  tokens per seconde
try3      with a mean time of   28.999  tokens per seconde
try5     with a mean time of   28.862  tokens per seconde
try2      with a mean time of   28.525  tokens per seconde
try4      with a mean time of   28.016  tokens per seconde
Threads 2     with a mean time of   27.175  tokens per seconde
Threads 1     with a mean time of   21.524  tokens per seconde
typicaly : 9      with a mean time of   18.939  tokens per seconde
typicaly : 5      with a mean time of   18.205  tokens per seconde
_np05_ns2_ps1_     with a mean time of   17.002  tokens per seconde
typicaly : 0      with a mean time of   16.039  tokens per seconde
Threads 4     with a mean time of   15.965  tokens per seconde
grp_att_width_256     with a mean time of   15.816  tokens per seconde
grp_att_factor_width     with a mean time of   14.963  tokens per seconde
grp_att_factor_2     with a mean time of   14.93  tokens per seconde
top-p : 5      with a mean time of

# 1) LLAMA without options

In [42]:
csv = "summerize.csv"
df_without_option = pd.read_csv(csv, sep=';', encoding='latin1')
df_without_option = df_without_option[df_without_option['Option'] == "without option"]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df_without_option.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df_without_option.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df_without_option.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df_without_option.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df_without_option.iloc[0,5]) + " Tokens per seconde")

Option : GGUFV3(latest)
mean_load_time : 943.806 ms
mean_sample_token_per_seconde : 10216.819 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 19.284 Tokens per seconde
mean_eval_time_token_per_seconde : 3.827 Tokens per seconde
mean_total_time_token_per_seconde : 4.454 Tokens per seconde


In [43]:
csv = "gpt2_test_input_only_aggregation.csv"
df_without_option_aggregation = pd.read_csv(csv, sep=';', encoding='latin1')
df_without_option_aggregation = df_without_option_aggregation[['load_time','sample_time','ms_per_token','token_per_sec','prompt_eval_time','ms_per_token.1','token_per_sec.1','eval_time','ms_per_token.2','token_per_sec.2','total_time','name_']]

In [44]:


path_input = "../inputs/"
path_output = "../outputs/gpt2_test_input_only/outputs_res/"


def input_output_f(df, path_output):
    path_input = "../inputs/"
    input_output = []
    for i in range(len(df)):
        input_tab= df.iloc[i,-1].split('_')

        path_input_tmp = path_input + input_tab[0] + ".txt"
        path_output_tmp = path_output+ ''.join(input_tab[:-1]) + "_res.txt"

        with open(path_input_tmp, "r") as f:
            prompt = f.read()
        with open(path_output_tmp, "r") as f:
            res = f.read()

        input_output.append([prompt,res])
    return input_output

def print_index(index):
    print(input_output_f(df_without_option_aggregation, path_output)[index])

def clean(df):
    df = df[['load_time','sample_time','ms_per_token','token_per_sec','prompt_eval_time','ms_per_token.1','token_per_sec.1','eval_time','ms_per_token.2','token_per_sec.2','total_time','name_']]
    #Calcule mean_load_time
    _load_time = df['load_time'].str.replace("ms", "").astype(float)
    #Calcule mean_sample_token_per_seconde
    _sample_token_per_seconde = df['token_per_sec'].astype(float)
    #Calcule mean_prompt_eval_time_token_per_seconde
    _prompt_eval_time_token_per_seconde = df['token_per_sec.1'].astype(float)
    #Calcule mean_eval_time_token_per_seconde
    _eval_time_token_per_seconde = df['token_per_sec.2'].astype(float)
    #Calcule mean_total_time_token_per_seconde
    step1_mean_total_time_token_per_seconde= df['total_time'].str.replace("ms", "").str.replace("tokens","")
    step2_1 = step1_mean_total_time_token_per_seconde.apply(lambda x : x.split('/')[0]).astype(float)
    step2_2 = step1_mean_total_time_token_per_seconde.apply(lambda x : x.split('/')[1]).astype(float)
    
    step3 = 1000/(step2_1 / step2_2)
    df['_load_time'] = _load_time
    df['_sample_token_per_seconde'] = _sample_token_per_seconde
    df['_prompt_eval_time_token_per_seconde'] = _prompt_eval_time_token_per_seconde
    df['total_time_token_per_seconde'] = step3
    
    return df[['name_', '_load_time', '_sample_token_per_seconde','_prompt_eval_time_token_per_seconde','total_time_token_per_seconde']]

def print_best(df):
    print("Best load_time (ms)")
    print(df.sort_values(by='_load_time').iloc[1,:])
    print('\n\n')


    print("Best _sample_token_per_seconde")
    print(df.sort_values(by='_sample_token_per_seconde', ascending=False).iloc[1,:])
    print('\n\n')


    print("Best _prompt_eval_time_token_per_seconde")
    print(df.sort_values(by='_prompt_eval_time_token_per_seconde', ascending=False).iloc[1,:])
    print('\n\n')


    print("Best total_time_token_per_seconde")
    print(df.sort_values(by='total_time_token_per_seconde', ascending=False).iloc[1,:])
    print('\n\n')


In [45]:
df_without_option = clean(df_without_option_aggregation)
print_best(df_without_option)

Best load_time (ms)
name_                                  input9_log.txt
_load_time                                     309.08
_sample_token_per_seconde                    10281.72
_prompt_eval_time_token_per_seconde             11.36
total_time_token_per_seconde                 2.016571
Name: 9, dtype: object



Best _sample_token_per_seconde
name_                                  input10_log.txt
_load_time                                      867.06
_sample_token_per_seconde                     11296.88
_prompt_eval_time_token_per_seconde              25.17
total_time_token_per_seconde                  3.209608
Name: 0, dtype: object



Best _prompt_eval_time_token_per_seconde
name_                                  input1_log.txt
_load_time                                    1131.45
_sample_token_per_seconde                     9004.14
_prompt_eval_time_token_per_seconde             28.97
total_time_token_per_seconde                 3.228248
Name: 1, dtype: object



Best total_time

In [46]:
print_index(0)

['What are the pros and cons of social media usage in modern society, and how can individuals and communities mitigate the negative effects?', '<|endoftext|>What are the pros and cons of social media usage in modern society, and how can individuals and communities mitigate the negative effects?\n\nIt is equally as important as developing any technology of our time. The world we live in today is a world that is fast growing and changing, and it is critical to meet these demands.\n\nI have had my own social media habits and the one I now have is social media and one of the best social media tools to be used.\n\nIn this tutorial I will show you how to create a social media tracking and social analytics tool. The tool is part of an on-']


In [47]:
df_without_option.sort_values(by='_load_time')

,name_,_load_time,_sample_token_per_seconde,_prompt_eval_time_token_per_seconde,total_time_token_per_seconde
6,input6_log.txt,172.12,8880.99,19.19,4.261193
9,input9_log.txt,309.08,10281.72,11.36,2.016571
7,input7_log.txt,316.17,11170.69,6.20,3.728021
8,input8_log.txt,446.39,9859.02,26.43,4.293110
2,input2_log.txt,584.48,8032.77,44.55,7.099963
3,input3_log.txt,829.74,11924.64,10.66,7.548488
0,input10_log.txt,867.06,11296.88,25.17,3.209608
1,input1_log.txt,1131.45,9004.14,28.97,3.228248
5,input5_log.txt,1896.97,11063.17,10.54,4.938679
4,input4_log.txt,2884.60,10654.17,9.77,4.214075


In [48]:
df_without_option.sort_values(by='_sample_token_per_seconde')

,name_,_load_time,_sample_token_per_seconde,_prompt_eval_time_token_per_seconde,total_time_token_per_seconde
2,input2_log.txt,584.48,8032.77,44.55,7.099963
6,input6_log.txt,172.12,8880.99,19.19,4.261193
1,input1_log.txt,1131.45,9004.14,28.97,3.228248
8,input8_log.txt,446.39,9859.02,26.43,4.293110
9,input9_log.txt,309.08,10281.72,11.36,2.016571
4,input4_log.txt,2884.60,10654.17,9.77,4.214075
5,input5_log.txt,1896.97,11063.17,10.54,4.938679
7,input7_log.txt,316.17,11170.69,6.20,3.728021
0,input10_log.txt,867.06,11296.88,25.17,3.209608
3,input3_log.txt,829.74,11924.64,10.66,7.548488


In [49]:
df_without_option.sort_values(by='_prompt_eval_time_token_per_seconde')

,name_,_load_time,_sample_token_per_seconde,_prompt_eval_time_token_per_seconde,total_time_token_per_seconde
7,input7_log.txt,316.17,11170.69,6.20,3.728021
4,input4_log.txt,2884.60,10654.17,9.77,4.214075
5,input5_log.txt,1896.97,11063.17,10.54,4.938679
3,input3_log.txt,829.74,11924.64,10.66,7.548488
9,input9_log.txt,309.08,10281.72,11.36,2.016571
6,input6_log.txt,172.12,8880.99,19.19,4.261193
0,input10_log.txt,867.06,11296.88,25.17,3.209608
8,input8_log.txt,446.39,9859.02,26.43,4.293110
1,input1_log.txt,1131.45,9004.14,28.97,3.228248
2,input2_log.txt,584.48,8032.77,44.55,7.099963


In [50]:
df_without_option.sort_values(by='total_time_token_per_seconde')

,name_,_load_time,_sample_token_per_seconde,_prompt_eval_time_token_per_seconde,total_time_token_per_seconde
9,input9_log.txt,309.08,10281.72,11.36,2.016571
0,input10_log.txt,867.06,11296.88,25.17,3.209608
1,input1_log.txt,1131.45,9004.14,28.97,3.228248
7,input7_log.txt,316.17,11170.69,6.20,3.728021
4,input4_log.txt,2884.60,10654.17,9.77,4.214075
6,input6_log.txt,172.12,8880.99,19.19,4.261193
8,input8_log.txt,446.39,9859.02,26.43,4.293110
5,input5_log.txt,1896.97,11063.17,10.54,4.938679
2,input2_log.txt,584.48,8032.77,44.55,7.099963
3,input3_log.txt,829.74,11924.64,10.66,7.548488


# 2) ctv_ctk

In [51]:
def KV(string):
    return "KV_" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(KV)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_ctv_ctk_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : KV_f32
mean_load_time : 330.722 ms
mean_sample_token_per_seconde : 12319.718 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 32.13 Tokens per seconde
mean_eval_time_token_per_seconde : 11.391 Tokens per seconde
mean_total_time_token_per_seconde : 12.766 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input8_ctk_ctv_f16_log.txt
_load_time                                                 141.35
_sample_token_per_seconde                                12876.64
_prompt_eval_time_token_per_seconde                         39.22
total_time_token_per_seconde                            27.167003
Name: 16, dtype: object



Best _sample_token_per_seconde
name_                                  input5_ctk_ctv_f16_log.txt
_load_time                                                 143.35
_sample_token_per_seconde                                13090.72
_prompt_eval_time_token_per_seco

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 3) Group attention

In [52]:
def in_(string):
    return "grp_att" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_grp_att_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : grp_att_factor_2
mean_load_time : 165.125 ms
mean_sample_token_per_seconde : 12501.543 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 35.58 Tokens per seconde
mean_eval_time_token_per_seconde : 13.423 Tokens per seconde
mean_total_time_token_per_seconde : 14.93 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input3_grp_att_n_log.txt
_load_time                                               136.88
_sample_token_per_seconde                              12150.67
_prompt_eval_time_token_per_seconde                       49.68
total_time_token_per_seconde                           9.460701
Name: 9, dtype: object



Best _sample_token_per_seconde
name_                                  input6_grp_att_w_log.txt
_load_time                                               155.58
_sample_token_per_seconde                               13328.0
_prompt_eval_time_token_per_seconde     

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 4) Keep

In [53]:
def in_(string):
    return "keep_" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_keep_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : keep_0
mean_load_time : 316.658 ms
mean_sample_token_per_seconde : 11689.775 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 29.813 Tokens per seconde
mean_eval_time_token_per_seconde : 7.986 Tokens per seconde
mean_total_time_token_per_seconde : 8.96 Tokens per seconde

 ############################################################## 



C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

Best load_time (ms)
name_                                  input10_keep_0_log.txt
_load_time                                             149.92
_sample_token_per_seconde                            11933.17
_prompt_eval_time_token_per_seconde                      29.3
total_time_token_per_seconde                         9.758405
Name: 0, dtype: object



Best _sample_token_per_seconde
name_                                  input1_keep_all_log.txt
_load_time                                              226.21
_sample_token_per_seconde                              13875.4
_prompt_eval_time_token_per_seconde                      43.02
total_time_token_per_seconde                         27.304018
Name: 5, dtype: object



Best _prompt_eval_time_token_per_seconde
name_                                  input6_keep_0_log.txt
_load_time                                            154.33
_sample_token_per_seconde                           12730.74
_prompt_eval_time_token_per_seconde             

# 5) Min-p

In [54]:
def in_(string):
    return "min-p" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_min-p_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : min-p 0.01
mean_load_time : 289.35 ms
mean_sample_token_per_seconde : 11115.159 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 29.864 Tokens per seconde
mean_eval_time_token_per_seconde : 7.911 Tokens per seconde
mean_total_time_token_per_seconde : 9.084 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input10_min_p_05_log.txt
_load_time                                               142.28
_sample_token_per_seconde                              12464.17
_prompt_eval_time_token_per_seconde                       27.43
total_time_token_per_seconde                           8.558754
Name: 1, dtype: object



Best _sample_token_per_seconde
name_                                  input5_min_p_05_log.txt
_load_time                                              227.45
_sample_token_per_seconde                             12782.82
_prompt_eval_time_token_per_seconde               

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 6) Mirostat

In [55]:
def in_(string):
    return "mirostat" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_mirostat_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : mirostat 1 / mirostat-lr 0.02 / mirostat-ent 15
mean_load_time : 630.363 ms
mean_sample_token_per_seconde : 120.991 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 27.774 Tokens per seconde
mean_eval_time_token_per_seconde : 7.643 Tokens per seconde
mean_total_time_token_per_seconde : 8.133 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input2_mirostat2_br02_ent15_log.txt
_load_time                                                           131.0
_sample_token_per_seconde                                            51.31
_prompt_eval_time_token_per_seconde                                  41.44
total_time_token_per_seconde                                      9.569991
Name: 64, dtype: object



Best _sample_token_per_seconde
name_                                  input4_mirostat1_br02_ent5_log.txt
_load_time                                                         403.17

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 7) NKVO

In [56]:
def in_(string):
    return "disable KV offload" in string or "Normal" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_nkvo_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : disable KV offload
mean_load_time : 1667.511 ms
mean_sample_token_per_seconde : 10516.053 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 23.552 Tokens per seconde
mean_eval_time_token_per_seconde : 4.479 Tokens per seconde
mean_total_time_token_per_seconde : 5.313 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input5_nkvo_log.txt
_load_time                                          175.96
_sample_token_per_seconde                         10844.81
_prompt_eval_time_token_per_seconde                  23.97
total_time_token_per_seconde                      4.006717
Name: 11, dtype: object



Best _sample_token_per_seconde
name_                                  input8_log.txt
_load_time                                    2958.84
_sample_token_per_seconde                    11675.42
_prompt_eval_time_token_per_seconde             14.87
total_time_token_per_seconde         

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 8) repeat_last_n

In [57]:
def in_(string):
    return "repeat last n" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_repeat_last_n_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : repeat last n : 0
mean_load_time : 398.63 ms
mean_sample_token_per_seconde : 11968.294 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 22.394 Tokens per seconde
mean_eval_time_token_per_seconde : 7.898 Tokens per seconde
mean_total_time_token_per_seconde : 8.913 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input4_repeat_last_all_log.txt
_load_time                                                     139.55
_sample_token_per_seconde                                    11866.62
_prompt_eval_time_token_per_seconde                             32.31
total_time_token_per_seconde                                  9.57201
Name: 19, dtype: object



Best _sample_token_per_seconde
name_                                  input5_repeat_last_32_log.txt
_load_time                                                    158.61
_sample_token_per_seconde                                   135

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 9) repeat_penalty

In [58]:
def in_(string):
    return "repeat penalty" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_repeat_penalty_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : repeat penalty : 1
mean_load_time : 1128.306 ms
mean_sample_token_per_seconde : 9888.492 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 22.673 Tokens per seconde
mean_eval_time_token_per_seconde : 4.536 Tokens per seconde
mean_total_time_token_per_seconde : 5.419 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input2_repeat_penalty__log.txt
_load_time                                                     136.83
_sample_token_per_seconde                                     1164.01
_prompt_eval_time_token_per_seconde                             28.87
total_time_token_per_seconde                                24.996623
Name: 11, dtype: object



Best _sample_token_per_seconde
name_                                  input1_repeat_penalty_1_log.txt
_load_time                                                       177.6
_sample_token_per_seconde                                

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 10) Split

In [59]:
def in_(string):
    return "_np" in string and "_ns" in string and "_ps" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_split_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

Option : _np01_ns10_ps10_
mean_load_time : 932.942 ms
mean_sample_token_per_seconde : 10503.113 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 20.035 Tokens per seconde
mean_eval_time_token_per_seconde : 6.428 Tokens per seconde
mean_total_time_token_per_seconde : 7.149 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input9_np05_ns2_ps1_log.txt
_load_time                                                  140.16
_sample_token_per_seconde                                 12874.98
_prompt_eval_time_token_per_seconde                          25.71
total_time_token_per_seconde                             10.934728
Name: 259, dtype: object



Best _sample_token_per_seconde
name_                                  input10_np05_ns1_ps2_log.txt
_load_time                                                   170.08
_sample_token_per_seconde                                  13670.54
_prompt_eva

# 11) Temp

In [60]:
def in_(string):
    return "temp" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_temp_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : temp : 0
mean_load_time : 459.572 ms
mean_sample_token_per_seconde : 8323.807 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 21.295 Tokens per seconde
mean_eval_time_token_per_seconde : 8.585 Tokens per seconde
mean_total_time_token_per_seconde : 9.493 Tokens per seconde

 ############################################################## 



C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

Best load_time (ms)
name_                                  input4temp05_log.txt
_load_time                                           158.81
_sample_token_per_seconde                          10395.01
_prompt_eval_time_token_per_seconde                   21.65
total_time_token_per_seconde                       8.014002
Name: 12, dtype: object



Best _sample_token_per_seconde
name_                                  input3temp5_log.txt
_load_time                                          664.54
_sample_token_per_seconde                         12365.52
_prompt_eval_time_token_per_seconde                  45.18
total_time_token_per_seconde                      9.381692
Name: 11, dtype: object



Best _prompt_eval_time_token_per_seconde
name_                                  input3temp5_log.txt
_load_time                                          664.54
_sample_token_per_seconde                         12365.52
_prompt_eval_time_token_per_seconde                  45.18
total_time_token_per_se

# 12) Tfs

In [61]:
def in_(string):
    return "tfs :" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_tfs_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : tfs : 0.1
mean_load_time : 1143.974 ms
mean_sample_token_per_seconde : 9333.044 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 16.632 Tokens per seconde
mean_eval_time_token_per_seconde : 2.894 Tokens per seconde
mean_total_time_token_per_seconde : 3.418 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input10_tfs_95_log.txt
_load_time                                             168.64
_sample_token_per_seconde                            11865.21
_prompt_eval_time_token_per_seconde                     22.91
total_time_token_per_seconde                         8.123619
Name: 5, dtype: object



Best _sample_token_per_seconde
name_                                  input5_tfs_95_log.txt
_load_time                                            738.35
_sample_token_per_seconde                           11676.79
_prompt_eval_time_token_per_seconde                    14.55
total

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

#  13) Thread

In [62]:
def in_(string):
    return "Threads " in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_thraed_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

Option : Threads 1
mean_load_time : 225.786 ms
mean_sample_token_per_seconde : 13850.154 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 30.543 Tokens per seconde
mean_eval_time_token_per_seconde : 20.29 Tokens per seconde
mean_total_time_token_per_seconde : 21.524 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input4_thread4_log.txt
_load_time                                             130.61
_sample_token_per_seconde                            12506.25
_prompt_eval_time_token_per_seconde                     21.38
total_time_token_per_seconde                         9.895663
Name: 15, dtype: object



Best _sample_token_per_seconde
name_                                  input5_thread1_log.txt
_load_time                                             203.38
_sample_token_per_seconde                            14566.64
_prompt_eval_time_token_per_seconde                     34.76

# 14) top -k 

In [63]:
def in_(string):
    return "top-k" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_top-p_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : top-k : 5 
mean_load_time : 652.963 ms
mean_sample_token_per_seconde : 14635.826 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 29.75 Tokens per seconde
mean_eval_time_token_per_seconde : 9.097 Tokens per seconde
mean_total_time_token_per_seconde : 10.304 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input10_top_p_01_log.txt
_load_time                                               144.74
_sample_token_per_seconde                               13466.2
_prompt_eval_time_token_per_seconde                       39.64
total_time_token_per_seconde                          24.793716
Name: 0, dtype: object



Best _sample_token_per_seconde
name_                                  input1_top_p_01_log.txt
_load_time                                               215.6
_sample_token_per_seconde                             13513.51
_prompt_eval_time_token_per_seconde              

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 15) top-p

In [64]:
def in_(string):
    return "top-p" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_top-p_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : top-p : 5 
mean_load_time : 627.645 ms
mean_sample_token_per_seconde : 12938.256 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 29.724 Tokens per seconde
mean_eval_time_token_per_seconde : 13.569 Tokens per seconde
mean_total_time_token_per_seconde : 14.807 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input10_top_p_01_log.txt
_load_time                                               144.74
_sample_token_per_seconde                               13466.2
_prompt_eval_time_token_per_seconde                       39.64
total_time_token_per_seconde                          24.793716
Name: 0, dtype: object



Best _sample_token_per_seconde
name_                                  input1_top_p_01_log.txt
_load_time                                               215.6
_sample_token_per_seconde                             13513.51
_prompt_eval_time_token_per_seconde            

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 16) Typical

In [65]:
def in_(string):
    return "typicaly" in string

csv = "summerize.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df = df[df['Option'].apply(in_)]
print("Option : " + str(df.iloc[0,0]))
print("mean_load_time : " + str(df.iloc[0,1]) + " ms")
print("mean_sample_token_per_seconde : " + str(df.iloc[0,2]) + " Tokens per seconde")
print("mean_prompt_eval_time_token_per_seconde : " + str(df.iloc[0,3]) + " Tokens per seconde")
print("mean_eval_time_token_per_seconde : " + str(df.iloc[0,4]) + " Tokens per seconde")
print("mean_total_time_token_per_seconde : " + str(df.iloc[0,5]) + " Tokens per seconde")
#print_index(0)
print("\n ############################################################## \n")
csv = "gpt2_test_input_typical_aggregation.csv"
df = pd.read_csv(csv, sep=';', encoding='latin1')
df_ = clean(df)
print_best(df_)

Option : typicaly : 0 
mean_load_time : 661.316 ms
mean_sample_token_per_seconde : 11811.744 Tokens per seconde
mean_prompt_eval_time_token_per_seconde : 35.932 Tokens per seconde
mean_eval_time_token_per_seconde : 14.107 Tokens per seconde
mean_total_time_token_per_seconde : 16.039 Tokens per seconde

 ############################################################## 

Best load_time (ms)
name_                                  input8_typical_9_log.txt
_load_time                                               130.89
_sample_token_per_seconde                              10519.67
_prompt_eval_time_token_per_seconde                       59.27
total_time_token_per_seconde                           7.584118
Name: 26, dtype: object



Best _sample_token_per_seconde
name_                                  input2_typical_0_log.txt
_load_time                                              1484.04
_sample_token_per_seconde                               12543.9
_prompt_eval_time_token_per_seconde     

C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_load_time'] = _load_time
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_sample_token_per_seconde'] = _sample_token_per_seconde
C:\Users\User\AppData\Local\Temp\ipykernel_23936\3361217310.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe